In [6]:
import yfinance as yf
import pandas as pd
import numpy as np

# Step 1: Download close prices
def download_close_prices(ticker, years):
    end_date = pd.Timestamp.today()
    start_date = end_date - pd.DateOffset(years=years)
    data = yf.download(ticker, start=start_date, end=end_date)
    return data['Close']

apple_close = download_close_prices('AAPL', 10)
google_close = download_close_prices('GOOGL', 10)
meta_close = download_close_prices('META', 10)

# Step 2: Merge dataframes on Date
merged_df = pd.concat([apple_close, google_close, meta_close], axis=1)
merged_df.columns = ['AAPL', 'GOOGL', 'META']

# Step 3: Calculate log returns
log_returns = np.log(merged_df / merged_df.shift(1)) * 100  #log returns multiplied by 100 to get in % values

# Step 4: Calculate VaR percentiles
def calculate_var(returns, confidence_level):
    return returns.quantile(1 - confidence_level)

confidence_levels = [0.99, 0.95, 0.90]
holding_periods = [1, 2, 5, 10]  # days

# Create a dictionary to store VaR results
var_results = {period: {} for period in holding_periods}
for stock in log_returns.columns:
    for period in holding_periods:
        var_results[period][stock] = {confidence_level: calculate_var(log_returns[stock].rolling(period).sum(), confidence_level) for confidence_level in confidence_levels}

# Create the summary dataframe
data = []
for period in holding_periods:
    row = {'Holding Period': f'{period} day'}
    for confidence_level in confidence_levels:
        for stock in log_returns.columns:
            var_value = var_results[period][stock][confidence_level]
            row[f'{stock} - {confidence_level * 100}% VaR'] = f'{var_value:.2f}%'  
    data.append(row)

summary_df = pd.DataFrame(data)

# Print summary dataframe
print(summary_df)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
  Holding Period AAPL - 99.0% VaR GOOGL - 99.0% VaR META - 99.0% VaR  \
0          1 day           -4.92%            -4.82%           -6.33%   
1          2 day           -7.30%            -6.36%           -8.48%   
2          5 day          -10.28%           -10.01%          -14.68%   
3         10 day          -13.77%           -12.40%          -21.40%   

  AAPL - 95.0% VaR GOOGL - 95.0% VaR META - 95.0% VaR AAPL - 90.0% VaR  \
0           -2.71%            -2.65%           -3.53%           -1.90%   
1           -3.87%            -3.81%           -4.87%           -2.70%   
2           -5.83%            -5.77%           -7.39%           -4.06%   
3           -8.49%            -7.68%           -9.86%           -6.14%   

  GOOGL - 90.0% VaR META - 90.0% VaR  
0            -1.88%   